<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/PAXDEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-dateutil faker -q
!pip install geopy -q
!pip install colab-env -q
import colab_env

In [ ]:
#!cp -pr /content/gdrive/MyDrive/datasets/DM/recoverai.db /content/gdrive/MyDrive/datasets/DM/recoverai-backup.db

In [44]:
import os

if __name__ == "__main__":
    # Delete the database file if it exists
    database_file = '/content/gdrive/MyDrive/datasets/DM/recoverai.db'
    if os.path.exists(database_file):
        try:
            os.remove(database_file)
            print(f"RecoverAI Agent: Existing database file '{database_file}' deleted.")
        except OSError as e:
            print(f"RecoverAI Agent: Error deleting database file '{database_file}': {e}")
            # Consider raising the exception or handling it appropriately based on your requirements
    else:
        print(f"RecoverAI Agent: Database file '{database_file}' not found.")



!cp -pr /content/gdrive/MyDrive/datasets/DM/recoverai-backup.db /content/gdrive/MyDrive/datasets/DM/recoverai.db


import random
import sqlite3
from datetime import datetime, timedelta
from faker import Faker

# --- RecoverAI (RAI) Profile ---
RAI_PROFILE = {
    "airline_name": "Sun Airlines",
    "aircraft_types": ["A320", "B737", "B787", "B777"],
    "crew_roles": ["CP", "FO", "FA"],  # Captain, First Officer, Flight Attendant
    "connection_times": {
        "domestic": timedelta(minutes=15),
        "international": timedelta(minutes=30)
    },
    "long_haul_augmented_crew": True,  # Details to be defined later
    "reserve_crew_and_aircraft": True,
    "crew_round_trip": True,
    "protect_pnr": True,
    "pnr_seat_assignment": True,
    "airports": {
        "domestic": ["JFK", "LAX", "ORD", "ATL", "DFW", "DEN", "SFO", "SEA", "MIA"],
        "international": ["HND", "ICN", "PVG", "HKG", "SIN", "LHR", "CDG", "FRA", "AMS", "MAD", "CUN", "PUJ", "MBJ", "SJU", "NAS"]
    },
    "home_bases": {
        "domestic": "DEN",
        "asia": "PVG",
        "europe": "FRA",
        "caribbean": "CUN"
    },
    "maintenance_bases": ["DEN", "JFK", "ATL", "ORD", "PVG", "FRA", "CUN"],
    "capacities": {
        "aircraft": {
            "A320": {"BC": 8, "EC": 150},
            "B737": {"BC": 10, "EC": 162},
            "B787": {"BC": 28, "EC": 269},
            "B777": {"BC": 40, "EC": 350}
        },
        "crew": {
            "A320": {"CP": 1, "FO": 1, "FA": 3},
            "B737": {"CP": 1, "FO": 1, "FA": 4},
            "B787": {"CP": 1, "FO": 1, "FA": 6},
            "B777": {"CP": 1, "FO": 1, "FA": 8}
        }
    }
}


# --- Database Setup and Data Generation ---
def create_tables():
    conn = sqlite3.connect('/content/gdrive/MyDrive/datasets/DM/recoverai.db')
    cursor = conn.cursor()

    # Flights table (modified)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS flights (
            flight_number TEXT PRIMARY KEY,
            origin TEXT,
            destination TEXT,
            departure_time TEXT,
            arrival_time TEXT,
            aircraft_id TEXT,
            aircraft_type TEXT,
            status TEXT,
            cost REAL,
            distance REAL,
            FOREIGN KEY (aircraft_id) REFERENCES aircraft(aircraft_id)
        )
    """)

    cursor.execute("CREATE INDEX IF NOT EXISTS idx_flights_origin ON flights (origin)")
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_flights_destination ON flights (destination)")
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_flights_departure_time ON flights (departure_time)")

    # OAG Flights table
    cursor.execute("""
         CREATE TABLE IF NOT EXISTS oag_flights (
             flight_number TEXT PRIMARY KEY,
             origin TEXT,
             destination TEXT,
             departure_time TEXT,
             arrival_time TEXT,
             aircraft_type TEXT,
             status TEXT,
             cost REAL,
             distance REAL
         )
     """)

    cursor.execute("CREATE INDEX IF NOT EXISTS idx_oag_flights_origin ON oag_flights (origin)")
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_oag_flights_destination ON oag_flights (destination)")
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_oag_flights_departure_time ON oag_flights (departure_time)")

    # Passengers table
    cursor.execute("""
       CREATE TABLE IF NOT EXISTS passengers (
           passenger_id INTEGER PRIMARY KEY AUTOINCREMENT,
           name TEXT,
           flight_number TEXT,
           seat_number TEXT,
           pnr TEXT,
           class_of_service TEXT,
           ticket_type TEXT,
           FOREIGN KEY (flight_number) REFERENCES flights(flight_number)
       )
   """)

    # Crew table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS crew (
            crew_id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT,
            role TEXT,
            flight_number TEXT,
            is_reserve BOOLEAN,
            FOREIGN KEY (flight_number) REFERENCES flights(flight_number)
        )
    """)

    # Seat assignments table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS seat_assignments (
            seat_assignment_id INTEGER PRIMARY KEY AUTOINCREMENT,
            flight_number TEXT,
            seat_number TEXT,
            passenger_id INTEGER,
            class_of_service TEXT,
            FOREIGN KEY (flight_number) REFERENCES flights(flight_number),
            FOREIGN KEY (passenger_id) REFERENCES passengers(passenger_id)
        )
    """)

    # Aircraft table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS aircraft (
            aircraft_id TEXT PRIMARY KEY,
            type TEXT,
            capacity INTEGER,
            is_reserve BOOLEAN
        )
    """)

    conn.commit()
    conn.close()


from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy.distance import geodesic
import functools

@functools.lru_cache(maxsize=128)  # Cache results for up to 128 locations
def calculate_distance(origin, destination):
    """Calculates the distance between two airports using GeoPy with rate limiting and caching."""
    geolocator = Nominatim(user_agent="recoverai", timeout=10)  # Increased timeout to 10 seconds
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)  # Rate limiting: 1 second between requests

    try:
        location_origin = geocode(origin)
        location_destination = geocode(destination)

        if location_origin and location_destination:
            distance = geodesic((location_origin.latitude, location_origin.longitude),
                                (location_destination.latitude, location_destination.longitude)).km
            return distance
        else:
            print(f"RecoverAI Agent: Could not find location data for {origin} or {destination}.")
            return None  # Or handle the error appropriately

    except Exception as e:
        print(f"RecoverAI Agent: Error calculating distance: {e}")
        return None  # Or handle the error appropriately

def generate_data():
    """Generates sample data for flights, passengers, crew, and aircraft."""
    conn = sqlite3.connect('/content/gdrive/MyDrive/datasets/DM/recoverai.db')
    cursor = conn.cursor()
    faker = Faker()

    # --- Generate aircraft ---
    existing_aircraft_ids = set()
    for i in range(50):
        while True:
            aircraft_type = random.choice(RAI_PROFILE["aircraft_types"])
            aircraft_id = f"{aircraft_type}-{random.randint(1, 50):03}"
            if aircraft_id not in existing_aircraft_ids:
                existing_aircraft_ids.add(aircraft_id)
                break

        capacity = RAI_PROFILE["capacities"]["aircraft"][aircraft_type]["EC"] + RAI_PROFILE["capacities"]["aircraft"][aircraft_type]["BC"]
        # Some aircraft are designated as reserves
        is_reserve = i % 10 == 0  # Example: Every 10th aircraft is a reserve

        cursor.execute("INSERT INTO aircraft VALUES (?, ?, ?, ?)", (aircraft_id, aircraft_type, capacity, is_reserve))


    # --- Generate OAG flights with random airline codes ---
    airline_codes = ["AA", "DL", "UA", "BA", "AF", "LH", "JL", "KE", "QF", "EK", "CA", "NH", "SQ"]
    existing_flight_numbers = set()
    all_airports = RAI_PROFILE["airports"]["domestic"] + RAI_PROFILE["airports"]["international"]

    for i in range(1200):
        while True:
            flight_number = f"{random.choice(airline_codes)}{random.randint(1, 9999):04}"
            if flight_number not in existing_flight_numbers:
                existing_flight_numbers.add(flight_number)
                break

        origin = random.choice(all_airports)
        destination = random.choice([a for a in all_airports if a != origin])
        departure_time = faker.date_time_between(start_date="-1d", end_date="+1d")
        arrival_time = departure_time + timedelta(hours=random.randint(2, 12))

        # Determine if the flight is domestic or international
        is_domestic = origin in RAI_PROFILE["airports"]["domestic"] and destination in RAI_PROFILE["airports"]["domestic"]

        # Select aircraft type based on flight type (domestic or international)
        if is_domestic:
            cursor.execute("SELECT type FROM aircraft WHERE type IN (?, ?) ORDER BY RANDOM() LIMIT 1", ('A320', 'B737'))
        else:
            cursor.execute("SELECT type FROM aircraft WHERE type IN (?, ?) ORDER BY RANDOM() LIMIT 1", ('B787', 'B777'))

        aircraft_type = cursor.fetchone()[0]  # Get the selected aircraft type

        distance = calculate_distance(origin, destination) or 0  # Handle None values

        # Calculate cost using parameters (outside SQL query)
        fuel_cost_per_unit = 0.15  # Or get this value dynamically
        crew_cost_per_hour = 100  # Or get this value dynamically
        num_crew = sum(RAI_PROFILE["capacities"]["crew"].get(aircraft_type, {}).values())
        flight_duration_hours = (arrival_time - departure_time).total_seconds() / 3600
        cost = distance * fuel_cost_per_unit + num_crew * crew_cost_per_hour * flight_duration_hours

        status = "Scheduled"

        cursor.execute("""
            INSERT INTO oag_flights (flight_number, origin, destination, departure_time, arrival_time, aircraft_type, status, cost, distance)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (flight_number, origin, destination, departure_time, arrival_time, aircraft_type, status, cost, distance))

    # --- Generate Sun Airlines flights ---
    cursor.execute("SELECT aircraft_id FROM aircraft WHERE is_reserve = 0")
    available_aircraft_ids = [row[0] for row in cursor.fetchall()]

    for i in range(100):
        flight_number = f"SA{i + 1:03}"
        origin = random.choice(all_airports)
        destination = random.choice([a for a in all_airports if a != origin])
        departure_time = faker.date_time_between(start_date="-1d", end_date="+1d")
        arrival_time = departure_time + timedelta(hours=random.randint(2, 12))
        aircraft_id = random.choice(available_aircraft_ids)
        status = "Scheduled"

        # Calculate distance
        distance = calculate_distance(origin, destination) or 0  # Handle None values

        # Get aircraft type from aircraft_id
        aircraft_type = aircraft_id.split('-')[0]

        # Calculate cost using parameters
        fuel_cost_per_unit = 0.15  # Or get this value from a database or configuration
        crew_cost_per_hour = 100  # Or get this value from a database or configuration
        num_crew = sum(RAI_PROFILE["capacities"]["crew"].get(aircraft_type, {}).values())
        flight_duration_hours = (arrival_time - departure_time).total_seconds() / 3600
        cost = distance * fuel_cost_per_unit + num_crew * crew_cost_per_hour * flight_duration_hours


        # Insert into flights with parameterized query, accounting for aircraft_type
        cursor.execute("""
            INSERT INTO flights (flight_number, origin, destination, departure_time, arrival_time, aircraft_id, aircraft_type, status, cost, distance)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (flight_number, origin, destination, departure_time, arrival_time, aircraft_id, aircraft_type, status, cost, distance))



    # --- Generate passengers ---
    for i in range(500):  # Or however many passengers you are generating
        name = faker.name()
        flight_number = f"SA{random.randint(1, 100):03}"
        pnr = faker.bothify(text='?????-#####')
        class_of_service = random.choice(["EC", "BC"])
        ticket_type = "R"  # Example of adding a ticket_type value

        #print(f"About to insert passenger: {name}, {flight_number}, {None}, {pnr}, {class_of_service}, {ticket_type}")

        cursor.execute("INSERT INTO passengers VALUES (?, ?, ?, ?, ?, ?, ?)", (None, name, flight_number, None, pnr, class_of_service, ticket_type))



    # --- Generate crew ---
    roles = RAI_PROFILE["crew_roles"]
    for i in range(200):
        name = faker.name()
        role = random.choice(roles)
        flight_number = f"SA{random.randint(1, 100):03}"  # Assign crew to flights
        is_reserve = False  # Initially, all crew are not reserves

        # Assign some crew as reserves based on a percentage
        if random.random() < 0.1:  # 10% chance of being a reserve
            is_reserve = True
            flight_number = None  # Reserves are not assigned to a specific flight

        cursor.execute("INSERT INTO crew VALUES (?, ?, ?, ?, ?)", (None, name, role, flight_number, is_reserve))

    # --- Generate seat assignments ---
    cursor.execute("SELECT passenger_id, flight_number, class_of_service FROM passengers")
    passengers_data = cursor.fetchall()

    for passenger in passengers_data:
        passenger_id, flight_number, class_of_service = passenger

        aircraft_type_result = cursor.execute("""
            SELECT T2.type
            FROM flights AS T1
            INNER JOIN aircraft AS T2 ON T1.aircraft_id = T2.aircraft_id
            WHERE T1.flight_number = ?
        """, (flight_number,)).fetchone()

        if aircraft_type_result:
            aircraft_type = aircraft_type_result[0]
        else:
            print(f"Skipping seat assignment for passenger {passenger_id} on flight {flight_number} (Aircraft type not found)")
            continue  # Skip to the next passenger

        while True:
            if class_of_service == "BC":
                seat_number = f"{random.randint(1, RAI_PROFILE['capacities']['aircraft'][aircraft_type]['BC']):02}{chr(random.randint(65, 68))}"
            else:
                seat_number = f"{random.randint(1, RAI_PROFILE['capacities']['aircraft'][aircraft_type]['EC']):02}{chr(random.randint(65, 70))}"

            if not cursor.execute("SELECT 1 FROM seat_assignments WHERE flight_number = ? AND seat_number = ?", (flight_number, seat_number)).fetchone():
                break

        cursor.execute("INSERT INTO seat_assignments (flight_number, seat_number, passenger_id, class_of_service) VALUES (?, ?, ?, ?)",
                       (flight_number, seat_number, passenger_id, class_of_service))

    conn.commit()  # Commit the changes to the database
    conn.close()

# --- Cost Calculation and Resource Management ---

def calculate_flight_cost(flight_data):
    """Calculates the total cost for a flight."""
    aircraft_type = flight_data["aircraft_type"]
    distance = flight_data["distance"]

    # Fuel cost (simplified example)
    fuel_cost_per_unit = {
        "A320": 0.15,
        "B737": 0.18,
        "B787": 0.22,
        "B777": 0.25
    }
    fuel_cost = distance * fuel_cost_per_unit[aircraft_type]

    #print(f"Fuel cost for {aircraft_type} aircraft: {fuel_cost}")

    # Crew cost
    crew_cost_per_hour = 100
    num_crew = sum(RAI_PROFILE["capacities"]["crew"].get(aircraft_type, {}).values())  # Handle missing aircraft type
    flight_duration_hours = flight_data.get("flight_duration_hours", 5)  # Default to 5 hours if not provided
    crew_cost = num_crew * crew_cost_per_hour * flight_duration_hours


    # Airport fees (simplified example)
    airport_fees = 5000  # Placeholder airport fees

    # Maintenance cost (simplified example)
    maintenance_cost = 2000  # Placeholder maintenance cost

    total_cost = fuel_cost + crew_cost + airport_fees + maintenance_cost
    return total_cost

def assign_crew_and_aircraft(flight):
    """Assigns crew and aircraft to a flight, considering reserves and round-trips."""
    aircraft_type = flight["aircraft_type"]
    required_crew = RAI_PROFILE["capacities"]["crew"][aircraft_type]

    # Simplified crew assignment (replace with more detailed logic)
    crew = []
    for role, count in required_crew.items():
        for _ in range(count):
            # Find available crew with matching role (consider reserves and round-trips)
            # ... (Implement your crew selection logic here) ...
            crew.append({"name": "Crew Member", "role": role})  # Placeholder crew member

    # Simplified aircraft assignment (replace with more detailed logic)
    aircraft_id = f"{aircraft_type}-001"  # Placeholder aircraft ID
    # Check if aircraft is available and not under maintenance
    # ... (Implement your aircraft selection logic here) ...

    flight["crew"] = crew
    flight["aircraft_id"] = aircraft_id
    print(f"Assigned crew and aircraft to flight {flight['flight_number']}: {flight}")

def update_flight_schedule(flight):
    """Updates the flight schedule, considering connection times and airport constraints."""
    # ... (Implement your schedule update logic here) ...
    # Adjust departure and arrival times based on delays, cancellations, etc.
    # Consider connection times, airport curfews, and other constraints.
    print(f"Updated schedule for flight {flight['flight_number']}: {flight}")

# --- Helper Functions ---

def update_flight_status(flight_number, status):
    """Updates the status of a flight in the database."""
    conn = sqlite3.connect('/content/gdrive/MyDrive/datasets/DM/recoverai.db')
    cursor = conn.cursor()
    cursor.execute("UPDATE flights SET status = ? WHERE flight_number = ?", (status, flight_number))
    conn.commit()
    conn.close()

def get_passengers_on_flight(flight_number):
    """Retrieves passengers assigned to a flight."""
    conn = sqlite3.connect('/content/gdrive/MyDrive/datasets/DM/recoverai.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM passengers WHERE flight_number = ?", (flight_number,))
    passengers = cursor.fetchall()
    conn.close()
    return passengers



from datetime import timedelta

def calculate_flight_score(flight, original_flight):
    """Calculates a score for an alternative flight based on various factors."""


    # Ensure departure_time is datetime objects
    flight_departure_time = flight['departure_time']
    original_departure_time = original_flight['departure_time']

    # If they are strings, convert them to datetime objects
    if isinstance(flight_departure_time, str):
        flight_departure_time = datetime.strptime(flight_departure_time, '%Y-%m-%d %H:%M:%S.%f')
    if isinstance(original_departure_time, str):
        original_departure_time = datetime.strptime(original_departure_time, '%Y-%m-%d %H:%M:%S.%f')



    # Time difference (lower is better)
    time_diff = abs((flight_departure_time - original_departure_time).total_seconds() / 3600)
    time_diff_score = max(0, 100 - (time_diff * 10))  # Scale time difference, maximum score of 100

    # Cost difference (lower is better)
    cost_diff = abs(float(flight.get('cost', 0)) - float(original_flight.get('cost', 0)))  # Ensure both are floats
    cost_diff_score = max(0, 100 - (cost_diff / 10))  # Scale cost difference, maximum score of 100


    # Direct flight preference (direct flights get higher score)
    is_direct = flight.get('is_direct', True)  # Assume direct if not specified
    direct_flight_score = 100 if is_direct else 50

    # Airline preference (example: Sun Airlines preferred)
    is_sun_airlines = flight['flight_number'].startswith('SA')
    airline_preference_score = 80 if is_sun_airlines else 60

    # Connection time (if applicable, penalize short or very long connections)
    connection_time = flight.get('connection_time', None)
    connection_time_score = 100  # Default to 100 if no connection
    if connection_time:
        if connection_time < timedelta(hours=1):
            connection_time_score = 20  # Penalty for short connections
        elif connection_time > timedelta(hours=6):
            connection_time_score = 60  # Penalty for very long connections

    # Total score (weighted average)
    total_score = (
        0.3 * time_diff_score +
        0.2 * cost_diff_score +
        0.2 * direct_flight_score +
        0.15 * airline_preference_score +
        0.15 * connection_time_score
    )

    return total_score

#-----NEW-BEGIN--

from datetime import datetime, timedelta
import sqlite3

def sanitize_input(input_string):
    """Basic sanitization function to prevent SQL injection."""
    return input_string.replace("'", "''")

def get_aircraft_type(aircraft_id):
    """Helper function to retrieve aircraft type from the database."""
    conn = sqlite3.connect('/content/gdrive/MyDrive/datasets/DM/recoverai.db')
    cursor = conn.cursor()
    cursor.execute("SELECT type FROM aircraft WHERE aircraft_id = ?", (aircraft_id,))
    result = cursor.fetchone()
    conn.close()
    return result[0] if result else "A320"  # Default to A320 if not found

def get_passengers_on_flight_with_details(flight_number):
    """Retrieves passenger details for a given flight."""
    conn = sqlite3.connect('/content/gdrive/MyDrive/datasets/DM/recoverai.db')
    cursor = conn.cursor()
    cursor.execute("""
        SELECT passenger_id, name, class_of_service
        FROM passengers
        WHERE flight_number = ?
    """, (flight_number,))
    passengers = cursor.fetchall()
    conn.close()
    return passengers

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression  # Or any other suitable model

def predict_flight_disruption(flight_data):
    """Predicts the likelihood of a flight disruption using a machine learning model."""
    # 1. Load historical flight data (replace with your data source)
    historical_data = pd.read_csv("historical_flight_data.csv")

    # 2. Preprocess data (feature engineering, handling missing values, etc.)
    # ... (Implement your data preprocessing steps) ...

    # 3. Train a predictive model (example using Logistic Regression)
    X = historical_data[['departure_time', 'origin', 'destination', 'weather_conditions', '...']]  # Select relevant features
    y = historical_data['disrupted']  # Target variable (1 if disrupted, 0 otherwise)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # 4. Predict disruption probability for the given flight
    flight_features = pd.DataFrame([flight_data])  # Convert flight data to DataFrame
    disruption_probability = model.predict_proba(flight_features)[0][1]  # Probability of disruption (class 1)

    return disruption_probability


def generate_notification(passenger, flight, disruption_type=None):
    """Generates a personalized notification message."""

    # Get passenger details (e.g., name, frequent flyer status)
    passenger_name = passenger[1]  # Assuming name is in the second column of passenger data
    # ... (Get other relevant passenger details from your data structure) ...

    # Customize message based on disruption type and passenger details
    if disruption_type == "delay":
        message = f"Dear {passenger_name},\n\nWe regret to inform you that your flight {flight['flight_number']} has been delayed. "
        # ... (Add details about the delay, expected new departure time, etc.) ...
    elif disruption_type == "cancellation":
        message = f"Dear {passenger_name},\n\nWe regret to inform you that your flight {flight['flight_number']} has been canceled. "
        # ... (Add details about alternative options, rebooking instructions, etc.) ...
    else:  # Generic notification
        message = f"Dear {passenger_name},\n\nThere has been a change to your flight itinerary. "
        # ... (Add details about the new flight, departure time, etc.) ...

    # Add personalized elements based on passenger details
    # if frequent_flyer_status:
    #     message += f"As a valued {frequent_flyer_status} member, we are prioritizing your comfort and convenience."
    # ... (Add other personalized elements as needed) ...

    return message



def send_notification(passenger, flight, message, channel="email"):
    """Sends a notification to a passenger through the specified channel."""
    if channel == "email":
        # ... (Implement email sending logic here) ...
        # Example using smtplib (you'll need to configure your email settings):
        import smtplib
        from email.mime.text import MIMEText

        msg = MIMEText(message)
        msg['Subject'] = f"Flight Update: {flight['flight_number']}"
        msg['From'] = "your_airline_email@example.com"
        msg['To'] = passenger[7]  # Assuming passenger email is in the 8th column of passenger data

        with smtplib.SMTP('your_smtp_server', 587) as server:
            server.starttls()
            server.login("your_airline_email@example.com", "your_email_password")
            server.send_message(msg)

        print(f"Email notification sent to {passenger[1]} ({passenger[7]})")

    elif channel == "sms":
        # ... (Implement SMS sending logic here) ...
        # You can use a library like Twilio to send SMS messages.
        pass  # Placeholder for SMS logic

    # ... (Add support for other channels like mobile app push notifications) ...


#import your_llm_library  # Import your preferred LLM library

def generate_notification_with_llm(passenger, flight, disruption_type):
    """Generates a personalized message using an LLM assistant."""

    prompt = f"""
    Generate a personalized notification for {passenger[1]} about a flight {disruption_type} for flight {flight['flight_number']}.
    Include relevant details such as the new flight number (if applicable), departure time, and arrival time.
    Consider the passenger's frequent flyer status (if available) and travel class.
    Make the tone empathetic and informative.
    """  # Customize the prompt as needed
    notification_message = your_llm_library.generate_text(prompt)
    return notification_message

def find_alternative_flight(passenger, original_flight):
    """Finds an alternative flight, prioritizing based on a calculated score,
       considering Sun Airlines, reserve flights, and then OAG flights.
    """
    conn = sqlite3.connect('/content/gdrive/MyDrive/datasets/DM/recoverai.db')
    cursor = conn.cursor()

    # Sanitize input values
    origin = sanitize_input(original_flight['origin'])
    destination = sanitize_input(original_flight['destination'])

    # Relaxed time constraints (72 hours for both Sun Airlines and OAG)
    arrival_time_str = sanitize_input(str(original_flight['arrival_time'] + timedelta(hours=0)))
    end_time_str = sanitize_input(str(original_flight['arrival_time'] + timedelta(hours=24)))


    # --- 0. Proactive Disruption Prediction ---
    #disruption_risk = predict_flight_disruption(original_flight)  # Call prediction function
    #if disruption_risk > 0.7:  # Example threshold
    #    print(f"Warning: Flight {original_flight['flight_number']} has a high risk of disruption ({disruption_risk:.2f})")

    # --- 1. Prioritize Sun Airlines Flights (with Filtering) ---
    suitable_alternatives = []


    # Filter Sun Airlines flights based on criteria before calculating scores:
    query_sun_airlines = f"""
        SELECT flight_number, origin, destination, departure_time, arrival_time, aircraft_id, cost, distance
        FROM flights
        WHERE destination = '{destination}'
          AND origin = '{origin}'
          AND departure_time BETWEEN '{arrival_time_str}' AND '{end_time_str}'
          AND status = 'Scheduled'
          AND (SELECT COUNT(*) FROM passengers WHERE flight_number = flights.flight_number AND class_of_service = '{passenger[5]}') < (SELECT capacity FROM aircraft WHERE aircraft_id = flights.aircraft_id)  -- Check seat availability
    """
    cursor.execute(query_sun_airlines)


    # Process filtered results:
    for row in cursor.fetchall():
        flight = {
            'flight_number': row[0],
            'origin': row[1],
            'destination': row[2],
            'departure_time': datetime.strptime(row[3], "%Y-%m-%d %H:%M:%S.%f"),
            'arrival_time': datetime.strptime(row[4], "%Y-%m-%d %H:%M:%S.%f"),
            'aircraft_id': row[5],
            'cost': row[6],
            'distance': row[7],
            'is_oag': False,
            'aircraft_type': get_aircraft_type(row[5])
        }
        suitable_alternatives.append(flight)

    # Calculate cost for Sun Airlines alternatives (if not already present)
    for flight in suitable_alternatives:
        if 'calculated_cost' not in flight and not flight.get('is_oag', False):
            flight_data = {'aircraft_type': flight['aircraft_type'], 'distance': flight['distance']}
            flight['calculated_cost'] = calculate_flight_cost(flight_data)

    # --- Prioritization using flight scores ---
    scored_alternatives = [(flight, calculate_flight_score(flight, original_flight)) for flight in suitable_alternatives]
    scored_alternatives.sort(key=lambda item: item[1], reverse=True)
    best_alternative = scored_alternatives[0][0] if scored_alternatives else None


    # --- 2. Consider Connection Flights (if no suitable Sun Airlines flights found) ---
    if not best_alternative:
        print("RecoverAI Agent: Exploring connection flights...")
        max_layover_hours = 4

        cursor.execute(f"""
            -- ... (Your existing SQL query for connection flights) ...
        """)

        connection_alternatives = []
        for row in cursor.fetchall():
            # Extract data for the first flight leg
            flight1 = {
                'flight_number': row[0],
                'origin': row[1],
                'destination': row[2],  # This is the layover airport
                'departure_time': datetime.strptime(row[3], "%Y-%m-%d %H:%M:%S.%f"),
                'arrival_time': datetime.strptime(row[4], "%Y-%m-%d %H:%M:%S.%f"),
                'aircraft_type': row[10],
                'cost': row[11],
                'distance': row[12],
                'is_oag': False  # Assuming connection flights are within Sun Airlines
            }

            # Extract data for the second flight leg
            flight2 = {
                'flight_number': row[5],
                'origin': row[6],  # This is the layover airport
                'destination': row[7],  # This is the final destination
                'departure_time': datetime.strptime(row[8], "%Y-%m-%d %H:%M:%S.%f"),
                'arrival_time': datetime.strptime(row[9], "%Y-%m-%d %H:%M:%S.%f"),
                'aircraft_type': row[13],
                'cost': row[14],
                'distance': row[15],
                'is_oag': False  # Assuming connection flights are within Sun Airlines
            }

            # Combine flight legs into a connection itinerary
            connection = {
                'flights': [flight1, flight2],
                'total_cost': flight1['cost'] + flight2['cost'],
                'total_distance': flight1['distance'] + flight2['distance'],
                'layover_airport': flight1['destination'],
                'layover_duration': flight2['departure_time'] - flight1['arrival_time'],
                'is_oag': False,
                'is_direct': False
            }

            # Check real-time status for both flights (using placeholder)
            if get_flight_status(flight1['flight_number']) == "scheduled" and get_flight_status(flight2['flight_number']) == "scheduled":
                # Calculate connection score (adapt scoring logic for connections)
                connection['score'] = calculate_flight_score(connection, original_flight)
                connection_alternatives.append(connection)

        # Select the best connection alternative based on score
        if connection_alternatives:
            best_alternative = max(connection_alternatives, key=lambda conn: conn['score'])
            print(f"Best connection found: {best_alternative}")
        else:
            print("No suitable connection flights found.")



    # --- Consider reserve flights if no suitable Sun Airlines flights found ---
    if not best_alternative:
        min_threshold_hours = 3
        max_threshold_hours = 12
        try:
            cursor.execute("""
                SELECT flight_number, origin, destination, departure_time, arrival_time, aircraft_id, cost, distance
                FROM flights
                WHERE origin = ? AND destination = ? AND status = 'Reserve'
                AND ABS(strftime('%s', departure_time) - strftime('%s', ?)) BETWEEN ? AND ?
                ORDER BY ABS(strftime('%s', departure_time) - strftime('%s', ?))
            """, (original_flight['origin'], original_flight['destination'], original_flight['departure_time'],
                  min_threshold_hours * 3600, max_threshold_hours * 3600, original_flight['departure_time']))

            reserve_flight_data = cursor.fetchall()

            if reserve_flight_data:
                reserve_flights = [
                    {'flight_number': flight_data[0], 'origin': flight_data[1], 'destination': flight_data[2],
                     'departure_time': datetime.strptime(flight_data[3], "%Y-%m-%d %H:%M:%S.%f"),
                     'arrival_time': datetime.strptime(flight_data[4], "%Y-%m-%d %H:%M:%S.%f"),
                     'aircraft_id': flight_data[5], 'cost': flight_data[6], 'distance': flight_data[7],
                     'is_oag': False, 'aircraft_type': get_aircraft_type(flight_data[5])}
                    for flight_data in reserve_flight_data
                ]
                scored_reserve_flights = [(flight, calculate_flight_score(flight, original_flight)) for flight in reserve_flights]
                scored_reserve_flights.sort(key=lambda item: item[1], reverse=True)
                best_alternative = scored_reserve_flights[0][0] if scored_reserve_flights else None

                if best_alternative:
                    update_flight_status(best_alternative['flight_number'], "Scheduled")
            else:
                print("RecoverAI Agent: No Matching Reserve Flight found for passenger.")

        except sqlite3.Error as e:
            print(f"RecoverAI Agent: Error querying or processing reserve flight: {e}")

    # --- Consider OAG flights if still no suitable alternative ---
    if not best_alternative:  # Check if no Sun Airlines or reserve options are found
        query_oag = f"""
            SELECT flight_number, origin, destination, departure_time, arrival_time, aircraft_type, cost, distance
            FROM oag_flights
            WHERE destination = '{destination}'
            AND origin = '{origin}'
            AND ABS(strftime('%s', departure_time) - strftime('%s', '{str(original_flight["departure_time"])}')) < 7200
            ORDER BY ABS(strftime('%s', departure_time) - strftime('%s', '{str(original_flight["departure_time"])}'))
            LIMIT 1
        """
        #print(f"OAG Query: {query_oag}") # Print the OAG query
        cursor.execute(query_oag)
        oag_alternatives = [
            {'flight_number': row[0], 'origin': row[1], 'destination': row[2],
             'departure_time': datetime.strptime(row[3], "%Y-%m-%d %H:%M:%S.%f"),
             'arrival_time': datetime.strptime(row[4], "%Y-%m-%d %H:%M:%S.%f"),
             'aircraft_type': row[5], 'cost': row[6], 'distance': row[7], 'is_oag': True}
            for row in cursor.fetchall()
        ]

        if oag_alternatives:
            print(f"RecoverAI Agent: Found {len(oag_alternatives)} potential OAG alternatives.")  # Print statement 2
            print(f"OAG Alternatives: {oag_alternatives}")  # Print the OAG alternatives

            # Score and select the best alternative from oag_alternatives
            scored_oag_alternatives = [(flight, calculate_flight_score(flight, original_flight)) for flight in oag_alternatives]
            scored_oag_alternatives.sort(key=lambda item: item[1], reverse=True)  # Sort by score (descending)
            best_alternative = scored_oag_alternatives[0][0] if scored_oag_alternatives else None  # Select the highest scored flight

        else:
            print("RecoverAI Agent: No OAG flights found for a passenger.")


    # --- Passenger Communication ---
    #if best_alternative:  # If an alternative flight is found
    #    disruption_type = "delay"  # Or "cancellation" or other relevant type
    #    notification_message = generate_notification(passenger, best_alternative, disruption_type)
    #    send_notification(passenger, best_alternative, notification_message, channel="email")  # Or other preferred channel
    #else:
        # Handle the case where no suitable alternative is found
    #    print(f"RecoverAI Agent: No suitable alternative flight found for passenger {passenger[1]} on flight {original_flight['flight_number']}.")
        # You might want to send a notification to the passenger in this case as well.

    conn.close()
    return best_alternative

#----END------NEW-

def update_passenger_booking(passenger, new_flight, original_flight):
    """Updates a passenger's booking with a new flight, assigns a new seat, calculates delay, disruption cost (including hotel), and provides a detailed cost breakdown."""
    conn = sqlite3.connect('/content/gdrive/MyDrive/datasets/DM/recoverai.db')
    cursor = conn.cursor()

    # Update flight_number in passengers table
    cursor.execute("UPDATE passengers SET flight_number = ? WHERE passenger_id = ?", (new_flight['flight_number'], passenger[0]))

    # Generate new seat assignment
    aircraft_type = new_flight['aircraft_type']
    class_of_service = passenger[5]  # Get class of service from passenger data

    while True:
        if class_of_service == "BC":
            seat_number = f"{random.randint(1, RAI_PROFILE['capacities']['aircraft'][aircraft_type]['BC']):02}{chr(random.randint(65, 68))}"
        else:
            seat_number = f"{random.randint(1, RAI_PROFILE['capacities']['aircraft'][aircraft_type]['EC']):02}{chr(random.randint(65, 70))}"

        # Check for duplicate seat assignment
        if not cursor.execute("SELECT 1 FROM seat_assignments WHERE flight_number = ? AND seat_number = ?", (new_flight['flight_number'], seat_number)).fetchone():
            break

    # Update seat_number in passengers table
    cursor.execute("UPDATE passengers SET seat_number = ? WHERE passenger_id = ?", (seat_number, passenger[0]))

    conn.commit()
    conn.close()

    # Calculate delay time
    # Convert original_flight['departure_time'] to datetime before subtraction
    original_departure_time = datetime.strptime(original_flight['departure_time'], '%Y-%m-%d %H:%M:%S.%f')
    delay = new_flight['departure_time'] - original_departure_time


    # Calculate delay time (Corrected part)
    #original_departure_time = original_flight['departure_time']  # No need for strptime
    #delay = new_flight['departure_time'] - original_departure_time



    # Calculate disruption cost components
    original_flight_cost = original_flight.get('cost', 0)

    new_flight_cost = new_flight.get('cost', 0) if delay > timedelta(hours=12) else 0

    flight_cost_difference = new_flight_cost - original_flight_cost

    # Compensation (example: $100 for delays over 4 hours)
    compensation = 100 if delay > timedelta(hours=4) else 0

    # Hotel Cost (example: $150 per night if delay requires overnight stay)
    hotel_cost = 150 if delay > timedelta(hours=12) else 0

    # Total Disruption Cost (including hotel)
    disruption_cost = flight_cost_difference + compensation + hotel_cost

    print('\n')
    # Print flight details, seat assignment, delay, and disruption cost with details
    print(f"Updated booking for {passenger[1]} to flight {new_flight['flight_number']}")
    print(f"Departure Airport: {new_flight['origin']}, Departure Time: {new_flight['departure_time'].strftime('%Y-%m-%d %H:%M')}")
    print(f"Arrival Airport: {new_flight['destination']}, Arrival Time: {new_flight['arrival_time'].strftime('%Y-%m-%d %H:%M')}")
    print(f"New Seat Assignment: {seat_number}, Class of Service: {class_of_service}")
    print(f"Delay: {delay}")




    print('\n')
    # Print detailed cost breakdown (including hotel)
    print("\nDisruption Cost Details:")
    print(f"  Original Flight Cost: ${original_flight_cost:.2f}")
    print(f"  New Flight Cost: ${new_flight_cost:.2f}")
    print(f"  Flight Cost Difference: ${flight_cost_difference:.2f}")
    print(f"  Compensation: ${compensation:.2f}")
    print(f"  Hotel Cost: ${hotel_cost:.2f}")
    print(f"  Total Disruption Cost (including hotel): ${disruption_cost:.2f}")


def generate_notification(passenger, flight):
    """Generates a notification for a passenger."""
    # ... (Implement your notification logic here) ...
    # This function would likely generate a message with the updated flight details
    # and send it to the passenger via email, SMS, or other communication channels.
    print('\n')
    print(f"Notification for {passenger[1]}: Your new flight is {flight['flight_number']} departing at {flight['departure_time'].strftime('%Y-%m-%d %H:%M')}")

# --- Data Validation Functions ---

def validate_flight(flight):
    """Validates flight data for consistency."""
    errors = []
    if not flight['flight_number']:
        errors.append("Flight number is missing.")
    if flight['origin'] == flight['destination']:
        errors.append("Origin and destination cannot be the same.")
    # ... (Add other validation checks based on RAI_PROFILE) ...
    return errors



def validate_passenger(passenger):
    """Validates passenger data for consistency."""
    errors = []
    if not passenger['name']:
        errors.append("Passenger name is missing.")
    if not passenger['flight_number']:
        errors.append("Passenger flight number is missing.")
    # ... (Add other passenger validation checks based on RAI_PROFILE and your requirements) ...
    return errors

def validate_crew_member(crew_member):
    """Validates crew member data for consistency."""
    errors = []
    if not crew_member['name']:
        errors.append("Crew member name is missing.")
    if not crew_member['role']:
        errors.append("Crew member role is missing.")
    # ... (Add other crew member validation checks based on RAI_PROFILE and your requirements) ...
    return errors

def validate_seat_assignment(seat_assignment):
    """Validates seat assignment data for consistency."""
    errors = []
    if not seat_assignment['flight_number']:
        errors.append("Flight number is missing for seat assignment.")
    if not seat_assignment['seat_number']:
        errors.append("Seat number is missing for seat assignment.")
    if not seat_assignment['passenger_id']:
        errors.append("Passenger ID is missing for seat assignment.")
    # ... (Add other seat assignment validation checks based on RAI_PROFILE and your requirements) ...
    return errors




# --- Test Case Scenario ---

def suggest_alternatives(passenger, original_flight):
    """Suggests alternative flights with the same airports and closest departure time."""
    conn = sqlite3.connect('recoverai.db')
    cursor = conn.cursor()

    # Query OAG flights with the same origin and destination:
    cursor.execute("""
        SELECT flight_number, origin, destination, departure_time, arrival_time, aircraft_type, capacity
        FROM oag_flights
        WHERE origin = ? AND destination = ? AND status = 'Scheduled'
        ORDER BY ABS(strftime('%s', departure_time) - strftime('%s', ?))  -- Order by time difference
        LIMIT 10  -- Limit to 10 closest flights
    """, (original_flight['origin'], original_flight['destination'], original_flight['departure_time']))  # Pass original departure time

    oag_alternatives = [
        {'flight_number': row[0], 'origin': row[1], 'destination': row[2],
         'departure_time': datetime.strptime(row[3], "%Y-%m-%d %H:%M:%S.%f"),
         'arrival_time': datetime.strptime(row[4], "%Y-%m-%d %H:%M:%S.%f"),
         'aircraft_type': row[5], 'capacity': row[6]}
        for row in cursor.fetchall()
    ]

    # Present the alternatives to the passenger (using print statements for now)
    print("Alternative flights found (OAG):")
    for flight in oag_alternatives:
        print(
            f"  Flight: {flight['flight_number']}, Departure: {flight['departure_time'].strftime('%Y-%m-%d %H:%M')}, Arrival: {flight['arrival_time'].strftime('%Y-%m-%d %H:%M')}"
        )

    conn.close()



#--------
def get_passengers_on_flight_with_categories(flight_number):
    """Retrieves passenger details for a given flight and categorizes them based on service class,
       frequent flyer status, connecting flights, family status, and creates a general category.
    """
    conn = sqlite3.connect('/content/gdrive/MyDrive/datasets/DM/recoverai.db')
    cursor = conn.cursor()

    # Select all necessary passenger details from the passengers table
    cursor.execute("""
        SELECT passenger_id, name, class_of_service, flight_number, seat_number, pnr, ticket_type, 'passenger@example.com' -- Replace with actual email column
        FROM passengers
        WHERE flight_number = ?
    """, (flight_number,))

    passengers = cursor.fetchall()
    conn.close()

    categorized_passengers = {
        "frequent_flyers": [],  # Placeholder for frequent flyers (add logic as needed)
        "business_class": [],
        "connecting": [],       # Placeholder for connecting passengers (add logic as needed)
        "families": [],         # Placeholder for families (add logic as needed)
        "general": []
    }

    for passenger in passengers:
        if passenger[2] == "BC":  # Categorize as Business Class
            categorized_passengers["business_class"].append(passenger)
        # Add other categorization logic here (frequent flyers, connecting, families) as needed
        else:  # Default to general category
            categorized_passengers["general"].append(passenger)

    return categorized_passengers

def rebook_on_flight(passenger, replacement_flight):
    """Attempts to rebook a passenger on the replacement flight.

    Returns:
        True if rebooking is successful, False otherwise.
    """
    if replacement_flight is not None and check_seat_availability(replacement_flight, passenger[2]):  # Assuming passenger[2] is the class of service
        # Update passenger's flight number in the database (simplified example)
        conn = sqlite3.connect('/content/gdrive/MyDrive/datasets/DM/recoverai.db')
        cursor = conn.cursor()
        cursor.execute("UPDATE passengers SET flight_number = ? WHERE passenger_id = ?", (replacement_flight['flight_number'], passenger[0]))
        conn.commit()
        conn.close()
        print(f"RecoverAI Agent: Rebooked passenger {passenger[1]} on flight {replacement_flight['flight_number']}.")
        return True
    else:
        # Handle the case where replacement_flight is None
        print(f"RecoverAI Agent: No suitable replacement flight found for passenger {passenger[1]}.")
        return False  # or implement alternative handling

def check_seat_availability(flight, class_of_service):
    """Checks if seats are available on the flight for the given class of service.

    This is a simplified example. You need to implement your actual seat availability logic.

    Returns:
        True if seats are available, False otherwise.
    """
    # Replace this with your actual seat availability logic
    # Example: Assuming flights have a fixed capacity for each class
    if class_of_service == "BC":
        return flight.get('BC_capacity', 0) > 0  # Check if business class capacity is greater than 0
    else:  # Assuming "EC" or other classes
        return flight.get('EC_capacity', 0) > 0  # Check if economy class capacity is greater than 0


#--------


from datetime import datetime, timedelta
import sqlite3
from typing import List, Tuple, Any

# ... (Your other imports and RAI_PROFILE) ...

#def select_flight_and_passenger()-> List[Tuple[Any, ...]]:  # Return type is now only the list of passengers

from datetime import datetime, timedelta
import sqlite3
import random

# ... (Your other imports and functions like find_alternative_flight,
# update_passenger_booking, generate_notification, send_notification) ...

def select_flight_and_passengers()-> List[Tuple[Any, ...]]:
    """Selects flights to cancel and all passengers on those flights for rebooking."""
    conn = sqlite3.connect('/content/gdrive/MyDrive/datasets/DM/recoverai.db')
    cursor = conn.cursor()

    all_rebooked_passengers = []

    try:
        # 1. Select flights to cancel:
        cursor.execute("""
            SELECT flight_number, origin, destination, departure_time, arrival_time
            FROM flights
            WHERE origin = 'PVG'
            ORDER BY cost DESC
        """)
        flights_data = cursor.fetchall()

        if flights_data:
            for flight_data in flights_data:
                cancelled_flight_number, origin, destination, departure_time_str, arrival_time_str = flight_data

                # Parse departure and arrival times:
                departure_time = datetime.strptime(departure_time_str, "%Y-%m-%d %H:%M:%S.%f")
                arrival_time = datetime.strptime(arrival_time_str, "%Y-%m-%d %H:%M:%S.%f")

                original_flight = {
                    'flight_number': cancelled_flight_number,
                    'origin': origin,
                    'destination': destination,
                    'departure_time': departure_time,
                    'arrival_time': arrival_time
                }

                # 2. Select ALL passengers from the current cancelled flight:
                cursor.execute("SELECT * FROM passengers WHERE flight_number = ?", (cancelled_flight_number,))
                passengers_data = cursor.fetchall()

                if passengers_data:
                    print(f"Found {len(passengers_data)} passengers on flight {cancelled_flight_number} to rebook.")

                    # Process each passenger with try-except and detailed logging:
                    for passenger in passengers_data:
                        try:
                            print(f"Rebooking passenger: {passenger[1]} (PNR: {passenger[4]})")

                            # Detailed print statement for find_alternative_flight()
                            print("find_alternative_flight() called with:", passenger, original_flight)

                            alternative_flight = find_alternative_flight(passenger, original_flight)

                            if alternative_flight:
                                update_passenger_booking(passenger, alternative_flight, original_flight)
                                disruption_type = "delay"
                                notification_message = generate_notification(passenger, alternative_flight, disruption_type)
                                send_notification(passenger, alternative_flight, notification_message, channel="email")
                                all_rebooked_passengers.extend([passenger])
                            else:
                                print(f"No suitable alternative flight found for passenger {passenger[1]} (PNR: {passenger[4]}).")

                        except Exception as e:
                            print(f"Error processing passenger {passenger[1]} (PNR: {passenger[4]}): {e}")
                            # You might want to log the error to a file or database for further investigation

                else:
                    print(f"No passengers found on the cancelled flight {cancelled_flight_number}. Suggesting alternative flights...")
                    # You might want to call a function here to suggest alternative flights
                    # For example: suggest_alternatives(original_flight)

            # After the main logic, before returning:
            arrival_time_str = original_flight['arrival_time'].strftime("%Y-%m-%d %H:%M:%S")
            end_time_str = (original_flight['arrival_time'] + timedelta(hours=24)).strftime("%Y-%m-%d %H:%M:%S")

            return all_rebooked_passengers, original_flight, origin, destination, departure_time, arrival_time_str, end_time_str

        else:
            print("No flights found to cancel.")
            return [], {}, '', '', datetime.now(), '', ''  # Return empty values

    except sqlite3.Error as e:
        print(f"RecoverAI Agent: An error occurred: {e}")
        return [], {}, '', '', datetime.now(), '', ''  # Return empty values
    finally:
        conn.close()

def select_flight_and_passenger():
    """Selects a flight to cancel, passenger, and suggests alternatives if needed."""
    conn = sqlite3.connect('/content/gdrive/MyDrive/datasets/DM/recoverai.db')
    cursor = conn.cursor()

    try:


      # 1. Select a flight to cancel (originating from DEN):
        cursor.execute("""
            SELECT flight_number, origin, destination, departure_time, arrival_time
            FROM flights
            WHERE origin = 'PVG'
            ORDER BY cost DESC
            LIMIT 1
        """)  # Select by cost and retrieve origin/destination



        flight_data = cursor.fetchone()



        if flight_data:  # Check if flight_data is not None
              cancelled_flight_number, origin, destination, departure_time, arrival_time = flight_data

              original_flight = {
                'flight_number': cancelled_flight_number,
                'origin': origin,
                'destination': destination,
                'departure_time': departure_time,  # Store departure_time as datetime objec
                'arrival_time': datetime.strptime(arrival_time, '%Y-%m-%d %H:%M:%S.%f') # Assuming arrival_time is a string

              }



              # Sanitize values:
              origin = sanitize_input(original_flight['origin'])
              destination = sanitize_input(original_flight['destination'])
              # No need to create departure_time_str
              arrival_time_str = sanitize_input(str(original_flight['arrival_time'] + RAI_PROFILE['connection_times']['domestic']))
              end_time_str = sanitize_input(str(original_flight['arrival_time'] + timedelta(hours=24)))


              # Print flight details:
              print(f"Cancelling flight: {cancelled_flight_number}")
              print(f"Departure Airport: {origin}, Departure Time: {departure_time}")
              print(f"Arrival Airport: {destination}, Arrival Time: {arrival_time}")
              update_flight_status(cancelled_flight_number, "Canceled")
              print('\n')




        else:  # Handle the case where no flight was found
              print("No flights found to cancel.")
              return None, None



        # 2. Select a random passenger from the cancelled flight:
        cursor.execute("SELECT pnr FROM passengers WHERE flight_number = ? ORDER BY RANDOM() LIMIT 1", (cancelled_flight_number,))
        passenger_pnr_result = cursor.fetchone()

        if passenger_pnr_result:


            # Select ALL passengers from the current cancelled flight:
            cursor.execute("SELECT * FROM passengers WHERE flight_number = ?", (cancelled_flight_number,))
            passengers_data = cursor.fetchall()

            if passengers_data:
                print(f"Found {len(passengers_data)} passengers on flight {cancelled_flight_number} to rebook.")

            passenger_pnr = passenger_pnr_result[0]
            #print('\n')
            #print(f"Selected passenger for test case: (PNR: {passenger_pnr}, Flight: {cancelled_flight_number})")
            #print('\n')
            #return passenger_pnr, cancelled_flight_number  # Return the PNR and flight number



            # Return all required values:
            return (passenger_pnr, original_flight, origin, destination,
                        original_flight['departure_time'], arrival_time_str, end_time_str)  # Return departure_time directly




        else:
            # 3. If no passenger found, suggest alternative flights:
            print(f"No passengers found on the cancelled flight {cancelled_flight_number}. Suggesting alternative flights...")

            # Retrieve original flight data (including origin and departure_time):
            cursor.execute("SELECT origin, destination, departure_time, arrival_time FROM flights WHERE flight_number = ?", (cancelled_flight_number,))
            flight_data = cursor.fetchone()


            if flight_data:
                origin, destination, departure_time_str, arrival_time_str = flight_data
                # Check if departure_time_str and arrival_time_str are valid before parsing
                try:
                    departure_time = datetime.strptime(departure_time_str, "%Y-%m-%d %H:%M:%S.%f")
                    arrival_time = datetime.strptime(arrival_time_str, "%Y-%m-%d %H:%M:%S.%f")
                except (TypeError, ValueError):
                    print(f"Invalid departure or arrival time for flight {cancelled_flight_number}. Skipping alternative suggestions.")
                    return None, None

                original_flight = {
                    "flight_number": cancelled_flight_number,
                    "origin": origin,
                    "destination": destination,
                    "departure_time": departure_time,  # Include departure time
                    "arrival_time": arrival_time
                }

                # Suggest alternative flights using the updated suggest_alternatives function:
                suggest_alternatives(None, original_flight)  # Passenger is None as no passenger found
            else:
                print("Original flight data not found.")
                print(f"RecoverAI Agent: Original flight data not found for flight {cancelled_flight_number}. Cannot proceed with rebooking.")

            return None, None

    except sqlite3.Error as e:
        print(f"RecoverAI Agent: An error occurred: {e}")
        return None, None  # Return None in case of an error
    finally:
        conn.close()


def run_test_case(passenger_pnr, original_flight):
    """Runs a test case for the RecoverAI agent."""
    conn = sqlite3.connect('/content/gdrive/MyDrive/datasets/DM/recoverai.db')
    cursor = conn.cursor()

    try:
        # Get passenger details using PNR
        cursor.execute("SELECT * FROM passengers WHERE pnr = ?", (passenger_pnr,))
        passenger = cursor.fetchone()

        if passenger:
            print('\n')
            print(f"RecoverAI Agent: For Testing Select passenger with PNR: {passenger_pnr}")
            print(f"Passenger Name: {passenger[1]}")
            print(f"Passenger Category: {passenger[5]}")
            print('\n')

            # Display original flight details (using original_flight)
            print(f"Original Flight: {original_flight['flight_number']} from {original_flight['origin']} to {original_flight['destination']}")
            print(f"Departure Time: {original_flight['departure_time']}")
            print(f"Arrival Time: {original_flight['arrival_time']}")
            print('\n')
            # Call find_alternative_flight using passenger and original_flight
            alternative_flight = find_alternative_flight(passenger, original_flight)

            if alternative_flight:
                update_passenger_booking(passenger, alternative_flight, original_flight)
                generate_notification(passenger, alternative_flight)
            else:
                print("RecoverAI Agent: No suitable alternatives found for passenger.")

        else:
            print(f"RecoverAI Agent: Passenger with PNR '{passenger_pnr}' not found.")

    except sqlite3.Error as e:
        print(f"RecoverAI Agent: Error during test case: {e}")

    finally:
        conn.close()  # Always close the connection


# --- Main Execution ---
if __name__ == "__main__":

    print('\n')
    print("RecoverAI Agent: Creating tables...")
    #create_tables()

    print('\n')
    print("RecoverAI Agent: Generating data...")
    #generate_data()

    print('\n')
    print("RecoverAI Agent: Running test case scenario...")
    print('\n')

    (passenger_pnr, original_flight, origin, destination, departure_time,
    arrival_time_str, end_time_str) = select_flight_and_passenger()

    if passenger_pnr and original_flight:  # Use original_flight, not cancelled_flight_number
        #print(f"Testing recovery for passenger with PNR: {passenger_pnr}")
        # ... (Your recovery logic using passenger_pnr, original_flight, and other returned values) ...
        run_test_case(passenger_pnr, original_flight)  # Assuming this function now needs original_flight info
    else:
        print("Could not select a flight and passenger for the test case.")



RecoverAI Agent: Existing database file '/content/gdrive/MyDrive/datasets/DM/recoverai.db' deleted.


RecoverAI Agent: Creating tables...


RecoverAI Agent: Generating data...


RecoverAI Agent: Running test case scenario...


Cancelling flight: SA009
Departure Airport: PVG, Departure Time: 2025-02-01 11:04:47.613105
Arrival Airport: CUN, Arrival Time: 2025-02-01 22:04:47.613105


Found 5 passengers on flight SA009 to rebook.


RecoverAI Agent: For Testing Select passenger with PNR: wEbKa-21534
Passenger Name: Sara Vasquez
Passenger Category: BC


Original Flight: SA009 from PVG to CUN
Departure Time: 2025-02-01 11:04:47.613105
Arrival Time: 2025-02-01 22:04:47.613105


RecoverAI Agent: Exploring connection flights...
No suitable connection flights found.
RecoverAI Agent: No Matching Reserve Flight found for passenger.
RecoverAI Agent: Found 1 potential OAG alternatives.
OAG Alternatives: [{'flight_number': 'JL0353', 'origin': 'PVG', 'destination': 'CUN', 'departure_time': datetime.dat

In [ ]:
!cp -pr /content/gdrive/MyDrive/datasets/DM/recoverai.db /content/gdrive/MyDrive/datasets/DM/recoverai-backup.db

NUMBER OF GLOBAL RECORDS

In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('/content/gdrive/MyDrive/datasets/DM/recoverai.db')
cursor = conn.cursor()

# Get the number of records in each table
tables = ['passengers', 'flights', 'seat_assignments', 'crew', 'aircraft', 'oag_flights']

for table in tables:
    cursor.execute(f"SELECT COUNT(*) FROM {table}")
    count = cursor.fetchone()[0]
    print(f"Number of records in {table}: {count}")

# Close the connection
conn.close()

Number of records in passengers: 500
Number of records in flights: 100
Number of records in seat_assignments: 500
Number of records in crew: 200
Number of records in aircraft: 50
Number of records in oag_flights: 1200


In [ ]:
import sqlite3

def find_missing_passenger():
    conn = sqlite3.connect('/content/gdrive/MyDrive/datasets/DM/recoverai.db')
    cursor = conn.cursor()

    cursor.execute("""
        SELECT p.passenger_id, p.name, p.flight_number
        FROM passengers p
        LEFT JOIN seat_assignments sa ON p.passenger_id = sa.passenger_id
        WHERE sa.passenger_id IS NULL;
    """)

    missing_passenger = cursor.fetchone()
    conn.close()

    if missing_passenger:
        print(f"Missing passenger: {missing_passenger}")
        passenger_id, name, flight_number = missing_passenger
        investigate_missing_assignment(passenger_id, name, flight_number)
    else:
        print("All passengers have seat assignments.")

def investigate_missing_assignment(passenger_id, name, flight_number):
    conn = sqlite3.connect('/content/gdrive/MyDrive/datasets/DM/recoverai.db')
    cursor = conn.cursor()

    # 1. Check aircraft capacity
    cursor.execute("""
        SELECT T2.type, T2.capacity
        FROM flights AS T1
        INNER JOIN aircraft AS T2 ON T1.aircraft_id = T2.aircraft_id
        WHERE T1.flight_number = ?
    """, (flight_number,))
    aircraft_data = cursor.fetchone()

    if aircraft_data:
        aircraft_type, capacity = aircraft_data
        print(f"Passenger {name} is on flight {flight_number} with aircraft type {aircraft_type} (capacity: {capacity}).")

        # Check available seats in the passenger's class of service (you'll need to add logic for this)
        # ...

    else:
        print(f"Flight {flight_number} does not have a matching aircraft type.")

    # 2. Check flight data (ensure valid aircraft_id)
    # ... (Add logic to check flight data)

    # 3. Check error logs (if you have any)
    # ... (Add logic to check error logs)

    conn.close()

if __name__ == "__main__":
    find_missing_passenger()

All passengers have seat assignments.


10 FIRST RECORDS PER TABLES

In [ ]:
import sqlite3

def display_table_data(tables):
    """Displays table name, column names, and the first 10 records."""

    try:
        with sqlite3.connect('/content/gdrive/MyDrive/datasets/DM/recoverai.db') as conn:
            cursor = conn.cursor()

            for table in tables:
                # Get column names
                cursor.execute(f"PRAGMA table_info({table})")
                columns = [column_info[1] for column_info in cursor.fetchall()]

                # Print table and column names
                print(f"\nTable: {table}")
                print(columns)  # Print column names in the second row

                # Get and print records
                cursor.execute(f"SELECT * FROM {table} LIMIT 10")
                records = cursor.fetchall()
                for record in records:
                    print(record)

    except sqlite3.Error as e:
        print(f"Error accessing database: {e}")

if __name__ == "__main__":
    tables_to_display = ['passengers', 'flights', 'oag_flights', 'seat_assignments', 'crew', 'aircraft']
    display_table_data(tables_to_display)

In [ ]:
import sqlite3

def display_flight_counts():
    """Displays the number of flights in both tables."""
    conn = sqlite3.connect('/content/gdrive/MyDrive/datasets/DM/recoverai.db')
    cursor = conn.cursor()

    try:
        # Get flight count from flights table
        cursor.execute("SELECT COUNT(*) FROM flights")
        flights_count = cursor.fetchone()[0]

        # Get flight count from oag_flights table
        cursor.execute("SELECT COUNT(*) FROM oag_flights")
        oag_flights_count = cursor.fetchone()[0]

        print(f"Number of flights in 'flights' table: {flights_count}")
        print(f"Number of flights in 'oag_flights' table: {oag_flights_count}")

    except sqlite3.Error as e:
        print(f"Error retrieving flight counts: {e}")
    finally:
        conn.close()

# Call the function to display the counts
display_flight_counts()

Number of flights in 'flights' table: 100
Number of flights in 'oag_flights' table: 1200


In [ ]:
def select_flight_and_passenger():
    """Selects a flight to cancel, passenger, and suggests alternatives if needed."""
    conn = sqlite3.connect('/content/gdrive/MyDrive/datasets/DM/recoverai.db')
    cursor = conn.cursor()

    try:
        # 1. Find a cancellable flight with a replacement in oag_flights
        cursor.execute("""
            SELECT f.flight_number, f.origin, f.destination, f.departure_time, f.arrival_time
            FROM flights f
            WHERE EXISTS (
                SELECT 1
                FROM oag_flights o
                WHERE o.origin = f.origin
                AND o.destination = f.destination
                AND ABS(strftime('%s', o.departure_time) - strftime('%s', f.departure_time)) < 7200  -- Departure time within 2 hours
            )
            ORDER BY f.cost DESC  -- Consider cost for cancellation priority (optional)
            LIMIT 10
        """)
        flight_data = cursor.fetchone()

        if flight_data:
            cancelled_flight_number, origin, destination, departure_time, arrival_time = flight_data
            print(f"RecoverAI Agent: Flight to cancel: {cancelled_flight_number} ({origin} to {destination})")
            # ... (Rest of the function logic remains the same) ...
        else:
            print("RecoverAI Agent: No suitable flight found for cancellation and replacement.")

    except sqlite3.Error as e:
        print(f"RecoverAI Agent: Error selecting flight: {e}")
    finally:
        conn.close()

select_flight_and_passenger()

RecoverAI Agent: Flight to cancel: SA009 (PVG to CUN)


In [ ]:
def select_flight_and_passenger():
    """Selects a flight to cancel, passenger, and suggests alternatives if needed."""
    conn = sqlite3.connect('/content/gdrive/MyDrive/datasets/DM/recoverai.db')
    cursor = conn.cursor()

    try:
        # 1. Find a cancellable flight with a replacement in oag_flights AND with passengers
        cursor.execute("""
            SELECT f.flight_number, f.origin, f.destination, f.departure_time, f.arrival_time
            FROM flights f
            WHERE EXISTS (
                SELECT 1
                FROM oag_flights o
                WHERE o.origin = f.origin
                AND o.destination = f.destination
                AND ABS(strftime('%s', o.departure_time) - strftime('%s', f.departure_time)) < 7200  -- Departure time within 2 hours
            )
            AND EXISTS (  -- Check for passengers on the flight
                SELECT 1
                FROM passengers p
                WHERE p.flight_number = f.flight_number
            )
            ORDER BY f.cost DESC  -- Consider cost for cancellation priority (optional)
            LIMIT 10
        """)
        flight_data = cursor.fetchone()

        if flight_data:
            cancelled_flight_number, origin, destination, departure_time, arrival_time = flight_data
            print(f"RecoverAI Agent: Flight to cancel: {cancelled_flight_number} ({origin} to {destination})")
            # ... (Rest of the function logic remains the same) ...
        else:
            print("RecoverAI Agent: No suitable flight found for cancellation and replacement (or no passengers on the flight).")

    except sqlite3.Error as e:
        print(f"RecoverAI Agent: Error selecting flight: {e}")
    finally:
        conn.close()

select_flight_and_passenger()

RecoverAI Agent: Flight to cancel: SA009 (PVG to CUN)


In [ ]:
def select_flight_and_passenger():
    """Selects multiple flights to cancel, passenger, and suggests alternatives if needed."""
    conn = sqlite3.connect('/content/gdrive/MyDrive/datasets/DM/recoverai.db')
    cursor = conn.cursor()

    try:
        # 1. Find cancellable flights with replacements in oag_flights
        cursor.execute("""
            SELECT f.flight_number, f.origin, f.destination, f.departure_time, f.arrival_time
            FROM flights f
            WHERE EXISTS (
                SELECT 1
                FROM oag_flights o
                WHERE o.origin = f.origin
                AND o.destination = f.destination
                AND ABS(strftime('%s', o.departure_time) - strftime('%s', f.departure_time)) < 7200  -- Departure time within 2 hours
            )
            ORDER BY f.cost DESC  -- Consider cost for cancellation priority (optional)
            LIMIT 10  -- Limit to 10 flights for now, adjust as needed
        """)
        flight_data_list = cursor.fetchall()  # Fetch all matching flights

        if flight_data_list:
            for flight_data in flight_data_list:  # Iterate through the flights
                cancelled_flight_number, origin, destination, departure_time, arrival_time = flight_data
                print(f"RecoverAI Agent: Flight to cancel: {cancelled_flight_number} ({origin} to {destination})")
                # ... (Rest of the function logic for each flight remains the same) ...
        else:
            print("RecoverAI Agent: No suitable flights found for cancellation and replacement.")

    except sqlite3.Error as e:
        print(f"RecoverAI Agent: Error selecting flight: {e}")
    finally:
        conn.close()

select_flight_and_passenger()

RecoverAI Agent: Flight to cancel: SA009 (PVG to CUN)
RecoverAI Agent: Flight to cancel: SA071 (LHR to MIA)
RecoverAI Agent: Flight to cancel: SA001 (SJU to FRA)
RecoverAI Agent: Flight to cancel: SA091 (CUN to LHR)
RecoverAI Agent: Flight to cancel: SA046 (SJU to ATL)
RecoverAI Agent: Flight to cancel: SA012 (CDG to MIA)
RecoverAI Agent: Flight to cancel: SA045 (SIN to LAX)
RecoverAI Agent: Flight to cancel: SA005 (CDG to MAD)
RecoverAI Agent: Flight to cancel: SA053 (DFW to PUJ)
RecoverAI Agent: Flight to cancel: SA039 (ORD to PVG)


In [ ]:
import sqlite3

def find_similar_oag_flight(cancelled_flight_number='SA071'):
    """Finds a similar flight in oag_flights for the given cancelled flight."""
    conn = sqlite3.connect('/content/gdrive/MyDrive/datasets/DM/recoverai.db')
    cursor = conn.cursor()

    try:
        cursor.execute("""
            SELECT o.flight_number
            FROM oag_flights o
            WHERE o.origin = (SELECT origin FROM flights WHERE flight_number = ?)
              AND o.destination = (SELECT destination FROM flights WHERE flight_number = ?)
              AND ABS(strftime('%s', o.departure_time) - strftime('%s', (SELECT departure_time FROM flights WHERE flight_number = ?))) < 7200
        """, (cancelled_flight_number, cancelled_flight_number, cancelled_flight_number))

        similar_flight = cursor.fetchone()

        if similar_flight:
            print(f"RecoverAI Agent: Similar OAG flight found: {similar_flight[0]}")
            return similar_flight[0]
        else:
            print("RecoverAI Agent: No similar OAG flight found.")
            return None

    except sqlite3.Error as e:
        print(f"RecoverAI Agent: Error finding similar flight: {e}")
        return None

    finally:
        conn.close()

# Example usage:
similar_oag_flight = find_similar_oag_flight()
if similar_oag_flight:
    print(f"The similar flight in oag_flights is: {similar_oag_flight}")

RecoverAI Agent: Similar OAG flight found: SQ3371
The similar flight in oag_flights is: SQ3371
